# Hyperparameter Optimization for Stance Classification (OPTUNA)

The goal of this notebook is to optimize the macro F1-score of a stance classification model using hyperparameter optimization (HPO) techniques with Optuna.
We fine-tune our winner model (Best Performance) on a balanced dataset (Previously Augmented) of tweets labeled as support or oppose.

We incorporate:
- Bayesian Optimization (TPE Sampler) to efficiently explore the hyperparameter space.
- Early Stopping to prevent overfitting by stopping training.
- Pruning (Median Pruner) to terminate unpromising trials early and save GPU time.
- Evaluation Metrics: Accuracy, Precision, Recall, and F1-score (macro).

In [26]:
# Libraries
import os
import re
import numpy as np
import pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import torch
from tqdm.auto import tqdm
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
import warnings
import json
warnings.filterwarnings('ignore')

# Google Colab or not
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    path = "/content/drive/MyDrive/multimodal-argmining"
    os.chdir(path)
    print(f"Loading data from Google Drive: {path}")
else:
    path = "C:/Users/diego/Desktop/Master Neuro/M2/Intership_NLP/multimodal-argmining"
    os.chdir(path)
    print(f"Loading data locally from: {path}")


# GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU ready:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU detecting, using CPU.")

#If we use the augmented dataset:
augmented = True 

# Set Seed
seed=42
np.random.seed(seed)
torch.manual_seed(seed)

Loading data locally from: C:/Users/diego/Desktop/Master Neuro/M2/Intership_NLP/multimodal-argmining
No GPU detecting, using CPU.


In [27]:
# Load Dataset
if augmented:
  train_path = f"{path}/data/train_augmented_paraphrase.csv"
else:
  train_path = f"{path}/data/train.csv"
  
dev_path   = f"{path}/data/dev.csv"
test_path  = f"{path}/data/test.csv"

df_train = pd.read_csv(train_path)
df_dev   = pd.read_csv(dev_path)
df_test  = pd.read_csv(test_path)


# Map labels to ints
label2id = {"oppose": 0, "support": 1}

#Minimal Preprocessing
def clean_tweet(text):

    text = str(text)
    #URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Replace symbols
    # text = text.replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">")
    # # Remove multiple spaces
    # text = re.sub(r'\s+', ' ', text).strip()


    # # Split the words inside Hashtag (#ILoveMexico -> # I Love Mexico)
    # def separar_hashtag(match):
    #     hashtag = match.group(1)
    #     separado = re.sub(r'([a-z])([A-Z])', r'\1 \2', hashtag)
    #     return f"# {separado}"

    # text = re.sub(r"#(\w+)", separar_hashtag, text)

    return text


for df in [df_train, df_dev, df_test]:
    df["tweet_text"] = df["tweet_text"].apply(clean_tweet)
    df["label"] = df["stance"].map(label2id)


dataset_train = Dataset.from_pandas(df_train[["tweet_text","label"]])
dataset_dev   = Dataset.from_pandas(df_dev[["tweet_text","label"]])
dataset_test  = Dataset.from_pandas(df_test[["tweet_text","label"]])

In [28]:
# We define our Model
MODEL_NAME = "roberta-base"
MAX_LEN=105

In [29]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
print(f"Tokenizer loaded: {MODEL_NAME}")


# Tokenization Function for each model
def tokenize_dataset(dataset, tokenizer, max_length=MAX_LEN):

    def tokenize_batch(batch):
        return tokenizer(batch["tweet_text"],padding="max_length",truncation=True,max_length=max_length)

    #Tokenization
    tokenized = dataset.map(tokenize_batch, batched=True)

    #Dataset Format for PyTorch
    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    return tokenized

# Tokenize datasets with model tokenizer
train_dataset_tok = tokenize_dataset(dataset_train, tokenizer, MAX_LEN)
dev_dataset_tok = tokenize_dataset(dataset_dev, tokenizer, MAX_LEN)
test_dataset_tok = tokenize_dataset(dataset_test, tokenizer, MAX_LEN)

print(f"Tokenization complete")

Tokenizer loaded: roberta-base


Map: 100%|██████████| 300/300 [00:00<00:00, 3283.30 examples/s]

Tokenization complete


In [30]:
#Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# We define our Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1_macro = f1_score(labels, preds, average="macro")
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    return {"accuracy": acc,"f1": f1_macro,"precision": precision,"recall": recall}

### Hyperparameter Optimization - OPTUNA

In [32]:
# Objective Function for Hyperparameter Optimization (OPTUNA)
def objective(trial):

    # Hyperparameters to optimize
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 3e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)
    per_device_train_batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 8)
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.0, 0.2)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)

    # New Model for each trial
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2,
        hidden_dropout_prob=dropout,
        attention_probs_dropout_prob=dropout).to(device)

    # Training Arguments
    training_args = TrainingArguments(
        output_dir=f"./optuna-trial-{trial.number}",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        warmup_ratio=warmup_ratio,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        save_total_limit=1,
        report_to="none",
        logging_strategy="epoch"
    )

    # Trainer with Early Stopping
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_tok,
        eval_dataset=dev_dataset_tok,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Training loop with pruning (More efficient)
    for epoch in range(num_train_epochs):
        trainer.train()
        eval_results = trainer.evaluate(dev_dataset_tok)
        f1_macro = eval_results["eval_f1"]

        # We Report to Optuna
        trial.report(f1_macro, step=epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    # Final evaluation (F1 score)
    final_metrics = trainer.evaluate(dev_dataset_tok)
    return final_metrics["eval_f1"]

We use **TPESampler** because it efficiently explores the hyperparameter space by learning from previous trials. Unlike random search, it focuses on promising regions, increasing the chances of finding optimal hyperparameters faster and with fewer trials. It is especially useful for mixed spaces with continuous, integer, and categorical parameters.

In [ ]:
# We create and run the Optuna study
study = optuna.create_study(direction="maximize",
                            sampler=TPESampler(seed=seed),
                            pruner=MedianPruner(n_warmup_steps=2))


study.optimize(objective, n_trials=20, timeout=None) 

print("Best trial:")
print(study.best_trial.params)
print("Best F1:", study.best_trial.value)

[I 2025-10-28 12:12:00,423] A new study created in memory with name: no-name-2dfbc722-fa04-4e8c-963b-ca3acf3c1996
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


[W 2025-10-28 12:19:16,843] Trial 0 failed with parameters: {'learning_rate': 3.574712922600241e-06, 'weight_decay': 0.09507143064099162, 'batch_size': 8, 'num_train_epochs': 3, 'warmup_ratio': 0.011616722433639893, 'dropout': 0.2598528437324805} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\diego\anaconda3\envs\multimodal\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_14888\3866794121.py", line 50, in objective
    trainer.train()
  File "c:\Users\diego\anaconda3\envs\multimodal\lib\site-packages\transformers\trainer.py", line 2325, in train
    return inner_training_loop(
  File "c:\Users\diego\anaconda3\envs\multimodal\lib\site-packages\transformers\trainer.py", line 2674, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
  File "c:\Users\diego\anaconda3\envs\multimodal\l

In [ ]:
# Re-train the best model with optimal hyperparameters
best_params = study.best_trial.params
print("\n Retraining best model with optimal hyperparameters...")

best_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    hidden_dropout_prob=best_params["dropout"],
    attention_probs_dropout_prob=best_params["dropout"]
).to(device)

best_args = TrainingArguments(
    output_dir="./roberta_best",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    num_train_epochs=best_params["num_train_epochs"],
    weight_decay=best_params["weight_decay"],
    warmup_ratio=best_params["warmup_ratio"],
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1,
    report_to="none",
    logging_strategy="epoch"
)

final_trainer = Trainer(
    model=best_model,
    args=best_args,
    train_dataset=train_dataset_tok,
    eval_dataset=dev_dataset_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

final_trainer.train()
eval_results = final_trainer.evaluate(test_dataset_tok)
print("\n Final evaluation on test set:", eval_results)

In [ ]:
# Path Output
os.makedirs(f"{path}/experiments/text/HPO/", exist_ok=True)
output_dir = f"{path}/experiments/text/HPO/"

# Confusion Matrix
preds_output = final_trainer.predict(test_dataset_tok)
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=-1)
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["oppose", "support"])

# Export
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(cmap="Blues", values_format="d", ax=ax)
plt.title(f"Confusion Matrix_Optimized RoBERTa,f1_Score={eval_results['eval_f1']:.4f}")
save_path = os.path.join(output_dir,f"confusion_matrix_Model_Optimized.jpg")
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()



In [ ]:
# Export metrics to CSV
metrics_dict = eval_results
metrics_dict["best_trial_f1_dev"] = study.best_trial.value 
results_df = pd.DataFrame([metrics_dict])
results_df.to_csv(output_dir + "evaluation_results.csv", index=False)
print(f"Metrics saved to {output_dir}/evaluation_results.csv")
print(results_df)

In [ ]:
# Export best hyperparameters to JSON
best_hyperparams = study.best_trial.params
with open(f"{output_dir}/best_hyperparameters.json", "w") as f:
    json.dump(best_hyperparams, f, indent=4)
print(f"Best hyperparameters saved to {output_dir}/best_hyperparameters.json")
print(best_hyperparams)